# The Pagila Database

One of the best example databases out there is the <a href="https://dev.mysql.com/doc/sakila/en/">Sakila Database</a>, which was originally created by MySQL and has been open sourced under the terms of the BSD License.

The Sakila database is a nicely normalised schema modelling a DVD rental store, featuring things like films, actors, film-actor relationships, and a central inventory table that connects films, stores, and rentals.

There is a port of Sakila database within PostgreSQL environment and it is named <a href="https://wiki.postgresql.org/wiki/Sample_Databases">Pagila Database</a>. This ported database version is what we're using for this class.

<img width="1200px" src="https://user-images.githubusercontent.com/7065401/58504872-fa243b00-8161-11e9-85ed-4b7d8d7ce9f7.png" />

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Hands on! 

In [ ]:
# Do below only when user is in Google Colab
!pip install psycopg2-binary

# The rest can be done anywhere else (Google Colab or not)
import psycopg2 # This is python driver to connect to PostgreSQL database
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Allow display of all rows and columns in a Pandas dataframe
pd.set_option("display.max_rows", None, "display.max_columns", None)

%matplotlib inline

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Loading our data:

In [ ]:
# Function to automate and permit easier access to Postgre SQL database using Pandas built-in tool
def create_pandas_table(params, sql_query, index_col=None, parse_dates=None):
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    table = pd.read_sql_query(sql_query, conn, index_col, parse_dates)
    cur.close()
    conn.close()
    return table

In [ ]:
# Start by defining the arguments needed for create_pandas_table function (above)
params = {'host': 'financeacademy-ahm.c6uftkcaibeq.ap-southeast-1.rds.amazonaws.com', 'database': 'pagila', 'user': 'analyst', 'password': 'digital', 'port' : '5432'}
sql_query = '''
    SELECT
        rental.rental_id, rental.rental_date, rental.return_date,
        customer.last_name AS customer_lastname,
        store.store_id,
        city.city AS rental_store_city,
        film.title AS film_title, film.rental_duration AS film_rental_duration,
        film.rental_rate AS film_rental_rate, film.replacement_cost AS film_replacement_cost,
        film.rating AS film_rating
    FROM rental
    INNER JOIN customer ON rental.customer_id = customer.customer_id
    INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id
    INNER JOIN store ON inventory.store_id = store.store_id
    INNER JOIN address ON store.address_id = address.address_id
    INNER JOIN city ON address.city_id = city.city_id
    INNER JOIN film ON inventory.film_id = film.film_id
    ORDER BY rental_id;
'''
parse_dates=['rental_date', 'return_date']
index_col='rental_id'

# Now run the function after defining all its arguments / parameters
df = create_pandas_table(params, sql_query,index_col, parse_dates)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## The data at a glance:

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Numerical analysis and visualization

We'll analyze the `film_rental_rate` column:

In [ ]:
df['film_rental_rate'].describe()

In [ ]:
round(df['film_rental_rate'].mean(),2)

In [ ]:
round(df['film_rental_rate'].median(),2)

In [ ]:
df['film_rental_rate'].plot(kind='box', vert=False, figsize=(14,6))

In [ ]:
df['film_rental_rate'].plot(kind='density', figsize=(14,6)) # kde

In [ ]:
ax = df['film_rental_rate'].value_counts().plot(kind='bar', figsize=(14,6))
ax.set_ylabel('Number of Rentals')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Categorical analysis and visualization

We'll analyze the `rental_store_city` column:

In [ ]:
df['rental_store_city'].value_counts()

In [ ]:
df['rental_store_city'].value_counts().plot(kind='pie', figsize=(6,6))

In [ ]:
ax = df['rental_store_city'].value_counts().plot(kind='bar', figsize=(14,6))
ax.set_ylabel('Number of Rentals')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Column wrangling

We can also create new columns or modify existing ones.

### Add and calculate a new `rental_rate_return` column

We want to know the rental rate of return of each film. To do that we'll use this formula:

$$ rental\_gain\_return = \frac{film\_rental\_rate}{film\_replacement\_cost} * 100 $$

In [ ]:
df['rental_gain_return'] = df['film_rental_rate'] / df['film_replacement_cost'] * 100

df['rental_gain_return'].head()

In [ ]:
df['rental_gain_return'].plot(kind='density', figsize=(14,6))

In [ ]:
round(df['rental_gain_return'].mean(), 2)

In [ ]:
round(df['rental_gain_return'].median(),2)

In [ ]:
ax = df['rental_gain_return'].plot(kind='density', figsize=(14,6)) # kde
ax.axvline(df['rental_gain_return'].mean(), color='red')
ax.axvline(df['rental_gain_return'].median(), color='green')

> Each rental represents <b>13.6%</b> of film cost.

So <b>7.35</b> rentals are needed to recover film market price (`film_replacement_cost`)

In [ ]:
100 / 13.6

While in average each film is rented <b>16.74</b> times.

In [ ]:
df['film_title'].value_counts().mean()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Selection & Indexing:

### Get the rental records of the customer with lastname `HANSEN`

In [ ]:
df_export = df.loc[df['customer_lastname'] == 'HANSEN']

In [ ]:
df_export

In [ ]:
df_export.to_csv('hansen.csv', index=True)
!ls

In [ ]:
# Only run these lines when using Colab
# from google.colab import files
# files.download('hansen.csv')

### Create a list of all the films with the highest replacement cost

In [ ]:
df['film_replacement_cost'].max()

In [ ]:
df.loc[df['film_replacement_cost'] == df['film_replacement_cost'].max(), 'film_title'].unique()

### How many `PG` or `PG-13` rating films were rented?

In [ ]:
df.loc[(df['film_rating'] == 'PG') | (df['film_rating'] == 'PG-13')].shape[0]

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)